##Pytorch Transformer Initialization Tutorial 


In [ ]:
# !pip install pytorch-pretrained-bert

In [ ]:
# from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification

In [ ]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True, do_basic_tokenize=True)
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

In [ ]:
# import torch
# output_model_file = "my_own_model_file.bin"
# output_config_file = "my_own_config_file.json"
# output_vocab_file = "./"

# # Step 1: Save a model, configuration and vocabulary that you have fine-tuned

# # If we have a distributed model, save only the encapsulated model
# # (it was wrapped in PyTorch DistributedDataParallel or DataParallel)
# model_to_save = model.module if hasattr(model, 'module') else model

# torch.save(model_to_save.state_dict(), output_model_file)
# model_to_save.config.to_json_file(output_config_file)
# tokenizer.save_vocabulary(output_vocab_file)



In [ ]:
# from pytorch_pretrained_bert import BertConfig

In [ ]:
# config = BertConfig.from_json_file('my_own_config_file.json')
# model = BertForSequenceClassification(config, num_labels=2)
# state_dict = torch.load('my_own_model_file.bin')
# model.load_state_dict(state_dict)
# tokenizer = BertTokenizer('vocab.txt', do_lower_case=True)

In [ ]:
# !mv my_own_config_file.json config.json

In [ ]:
# !mv my_own_model_file.bin pytorch_model.bin

In [ ]:
# model1 = BertForNextSentencePrediction.from_pretrained('./')
# tokenizer1 = BertTokenizer.from_pretrained('./')

##installing OSCAR 

In [ ]:
#%cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
#!rm -r /content/drive/MyDrive/Oscar

In [ ]:
#!rm -r /content/drive/MyDrive/pytorch_transformers.egg-info

In [ ]:
#!rm -r 067923d3267325f525f4e46f357360c191ba562e.zip

In [ ]:
#!rm -r transformers-067923d3267325f525f4e46f357360c191ba562e

In [ ]:
#!git clone https://github.com/microsoft/Oscar.git

In [ ]:
%cd /content/drive/MyDrive/Oscar

!python setup.py build develop

!pip install -r requirements.txt

In [ ]:
from oscar.modeling.modeling_bert import BertImgForPreTraining

In [ ]:
 #%cd /content/drive/MyDrive/Oscar/transformers

In [ ]:
#!ls

if this directory is empty, go to https://github.com/huggingface/transformers/tree/067923d3267325f525f4e46f357360c191ba562e and dowload a zip version using wget, make sure to move the unzipped files to this directory

The zip download link is https://github.com/huggingface/transformers/archive/067923d3267325f525f4e46f357360c191ba562e.zip

In [ ]:
#%cd /content/drive/MyDrive

In [ ]:
#!wget https://github.com/huggingface/transformers/archive/067923d3267325f525f4e46f357360c191ba562e.zip 

In [ ]:
#!unzip 067923d3267325f525f4e46f357360c191ba562e.zip

In [ ]:
#!mv /content/drive/MyDrive/transformers-067923d3267325f525f4e46f357360c191ba562e/* /content/drive/MyDrive/Oscar/transformers/

In [ ]:
#!ls /content/drive/MyDrive/Oscar/transformers

In [ ]:
#!cd /content/drive/MyDrive/Oscar/transformers

In [ ]:
#!python3 /content/drive/MyDrive/Oscar/transformers/setup.py build develop

if FileNotFoundError: [Errno 2] No such file or directory: 'README.md', go to 'setup.py' and modify line 45:


change "README.md" to "/content/drive/MyDrive/Oscar/transformers/README.md"

In [ ]:
#!pip install -r /content/drive/MyDrive/Oscar/transformers/requirements.txt

##Vanilla OSCAR with classifier, whose calss is **BertImgForPreTraining**
##Vanilla OSCAR is stored in its **bert** attribute, whose class is  **BertImgModel**

In [ ]:
# !cd /content/drive/MyDrive

# !wget https://biglmdiag.blob.core.windows.net/oscar/pretrained_models/base-vg-labels.zip

# !unzip /content/drive/MyDrive/base-vg-labels.zip

# !mv /content/drive/MyDrive/base-vg-labels/vg-oscar /content/drive/MyDrive/2021_Hate_Speech_Detection/model_pth

In [ ]:
# from oscar.modeling.modeling_bert import BertImgForPreTraining
# from transformers.pytorch_transformers import BertConfig

# config = BertConfig.from_json_file('/content/drive/MyDrive/2021_Hate_Speech_Detection/model_pth/vg-oscar/config.json')
# model1 =   BertImgForPreTraining(config)
# state_dict = torch.load('/content/drive/MyDrive/2021_Hate_Speech_Detection/model_pth/vg-oscar/pytorch_model.bin', map_location=torch.device('cpu'))

# model1.load_state_dict(state_dict)
# model.eval()
# oscar_state_dict = model.bert.state_dict()
# torch.save(oscar_state_dict, oscar_state_dict_path)

##Load objects and features from VinVl



In [ ]:
import json
import os
import os.path as op
from collections import OrderedDict


def load_labelmap_file(labelmap_file):
    label_dict = None

    if labelmap_file.endswith('json'):
        label_dict = json.load(open(labelmap_file, 'r'))['label_to_idx']
        label_dict = {key:val-1 for key, val in label_dict.items()}
        return label_dict

    if labelmap_file is not None and op.isfile(labelmap_file):
        label_dict = OrderedDict()
        with open(labelmap_file, 'r') as fp:
            for line in fp:
                label = line.strip().split('\t')[0]
                if label in label_dict:
                    raise ValueError("Duplicate label " + label + " in labelmap.")
                else:
                    label_dict[label] = len(label_dict)
    return label_dict


#return dictionary of image names and corresponidng objects names strings
def get_objects(vinvl_prediction_dictionary, dataset_labelmap):
  #get objects from each vinvl prediction, and concatenate them to string
  objects_strings={}
  for img_name in list(vinvl_prediction_dictionary.keys()):
    object_string = ''
    for i in vinvl_prediction_dictionary[img_name]['labels']:
      box_feature = i
      object_string+=dataset_labelmap[i.item()]+' '

    objects_strings[img_name] = object_string
  return objects_strings

#get a dictionary of 2-D image feature tensors (n, 2054), n is the number of objects in an image
def get_img_features(vinvl_prediction_dictionary):
  feature_dict = {}
  for img_name in list(vinvl_prediction_dictionary.keys()):
    feature_list = []
    for a, b in zip(vinvl_prediction_dictionary[img_name]['box_features'], vinvl_prediction_dictionary[img_name]['spatial_features']):
      object_feature = torch.cat((a, b))
      feature_list.append(object_feature)
    
    feature_dict[img_name] = torch.stack(feature_list)

  return feature_dict

def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

In [ ]:
#args.max_seq_length = 128
def tensorize_example(tokenizer, text_a, text_b, img_feat,  cls_token_at_end=False, pad_on_left=False,
                cls_token='[CLS]', sep_token='[SEP]', pad_token=0,
                sequence_a_segment_id=0, sequence_b_segment_id=1,
                cls_token_segment_id=1, pad_token_segment_id=0,
                mask_padding_with_zero=True):
    max_seq_length = 128
    max_img_seq_length = 40
    tokens_a = tokenizer.tokenize(text_a)

    tokens_b = None
    if text_b:
        tokens_b = tokenizer.tokenize(text_b)
        # Modifies `tokens_a` and `tokens_b` in place so that the total length is less than the specified length.
        # Account for [CLS], [SEP], [SEP] with "- 3"
        _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
    else:
        # Account for [CLS] and     [SEP] with "- 2"
        if len(tokens_a) > max_seq_length - 2:
            tokens_a = tokens_a[:(max_seq_length - 2)]

    tokens = tokens_a + [sep_token]
    segment_ids = [sequence_a_segment_id] * len(tokens)

    if tokens_b:
        tokens += tokens_b + [sep_token]
        segment_ids += [sequence_b_segment_id] * (len(tokens_b) + 1)

    if cls_token_at_end:
        tokens = tokens + [cls_token]
        segment_ids = segment_ids + [cls_token_segment_id]
    else:
        tokens = [cls_token] + tokens
        segment_ids = [cls_token_segment_id] + segment_ids

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real tokens are attended to.
    input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

    # Zero-pad up to the sequence length.
    padding_length = max_seq_length - len(input_ids)
    if pad_on_left:
        input_ids = ([pad_token] * padding_length) + input_ids
        input_mask = ([0 if mask_padding_with_zero else 1] * padding_length) + input_mask
        segment_ids = ([pad_token_segment_id] * padding_length) + segment_ids
    else:
        input_ids = input_ids + ([pad_token] * padding_length)
        input_mask = input_mask + ([0 if mask_padding_with_zero else 1] * padding_length)
        segment_ids = segment_ids + ([pad_token_segment_id] * padding_length)


    # image features
    

    if img_feat.shape[0] > max_img_seq_length:
        img_feat = img_feat[0: max_img_seq_length, ]
        if max_img_seq_length > 0:
            input_mask = input_mask + [1 if mask_padding_with_zero else 0] * img_feat.shape[0]
            segment_ids += [sequence_b_segment_id] * img_feat.shape[0]
    else:
        if max_img_seq_length > 0:
            input_mask = input_mask + [1 if mask_padding_with_zero else 0] * img_feat.shape[0]
            segment_ids = segment_ids + [sequence_b_segment_id] * img_feat.shape[0]
        padding_matrix = torch.zeros((max_img_seq_length - img_feat.shape[0], img_feat.shape[1]))
        img_feat = torch.cat((img_feat, padding_matrix), 0)
        if max_img_seq_length > 0:
            input_mask = input_mask + ([0 if mask_padding_with_zero else 1] * padding_matrix.shape[0])
            segment_ids = segment_ids + [pad_token_segment_id] * padding_matrix.shape[0]

 

    return (torch.tensor(input_ids, dtype=torch.long).unsqueeze(0),
            torch.tensor(input_mask, dtype=torch.long).unsqueeze(0),
            torch.tensor(segment_ids, dtype=torch.long)[0:128].unsqueeze(0),
            img_feat.unsqueeze(0))



In [ ]:
import json
from transformers.pytorch_transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#input_ids = torch.tensor(tokenizer.encode("[CLS] Hello, my dog is cute")).unsqueeze(0)
# input_ids=input_ids.to(device)
vinvl_det_dict = torch.load('path_to_vinvl_output')
vinvl_det_dict.keys()
labelmap_file = 'path_to_vinvl_labelmap'
dataset_allmap = json.load(open(labelmap_file, 'r'))
dataset_labelmap = {int(val): key for key, val in dataset_allmap['label_to_idx'].items()}
img_feat = get_img_features(vinvl_det_dict)['selfie']
input = tensorize_example(tokenizer, 'This is a person', 'hair nose face shirt ear ear man eye eye nostril head mouth collar lip eyebrow forehead eyebrow chin lip neck background', img_feat,  cls_token_at_end=False, pad_on_left=False,
                cls_token='[CLS]', sep_token='[SEP]', pad_token=0,
                sequence_a_segment_id=0, sequence_b_segment_id=1,
                cls_token_segment_id=1, pad_token_segment_id=0,
                mask_padding_with_zero=True)

In [ ]:
class OSCARBinaryClassification(torch.nn.Module):
    def __init__(self, oscar, linear):
        super(OSCARBinaryClassification, self).__init__()
        self.l1 = oscar
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = linear
    
    def forward(self, input_0, input_1, input_2, input_3):
        _, output_1= self.l1(input_ids=input_0, attention_mask = input_1, token_type_ids=input_2, img_feats=input_3)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output


In [ ]:
linear = nn.Linear(768, 2, bias=True)
linear.load_state_dict(torch.load('path_to_oscar_binary_classifier'))
oscarb = OSCARBinaryClassification(model.to('cpu'), linear)
oscarb.to(device)
model.to(device)
model(input[0].to(device).long(), input[1][:, :128].to(device).long(), input[2].to(device).long(), input[3].to(device).long())[0].to('cpu')
oscarb(input[0].to(device), input[1][:, :128].to(device), input[2].to(device), input[3].to(device))[0].to('cpu')
model(input_ids=input[0].to(device), attention_mask = input[1].to(device), token_type_ids=input[2].to(device), img_feats=input[3].to(device))[1].to('cpu')

##Output OSCAR Tensors for Training， Inputs are from VinVl 

In [ ]:
%cd /content/drive/MyDrive/Oscar

!python setup.py build develop

!pip install -r requirements.txt

In [ ]:
import torch
from oscar.modeling.modeling_bert import BertImgModel
from transformers.pytorch_transformers import BertConfig
oscar_state_dict = torch.load('path_to_finetuned_oscar_weights')
config = BertConfig.from_json_file('path_to_oscar_configuration')
model = BertImgModel(config)
model.load_state_dict(oscar_state_dict)
model.eval()


In [ ]:
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

In [ ]:
import json
from transformers.pytorch_transformers import BertTokenizer
import os
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.debug.metrics as met
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.utils.utils as xu
import torchvision
from torchvision import datasets, transforms


device = xm.xla_device()
model.to(device)


labelmap_file = 'path_to_vinvl_labelmap_json'
dataset_allmap = json.load(open(labelmap_file, 'r'))
dataset_labelmap = {int(val): key for key, val in dataset_allmap['label_to_idx'].items()}

def get_objects(vinvl_entry, dataset_labelmap):
  #get objects from each vinvl prediction, and concatenate them to string
  object_string = ''
  for i in vinvl_entry['labels']:
    box_feature = i
    object_string+=dataset_labelmap[i.item()]+' '
  return object_string

def get_img_features(vinvl_entry):
  feature_list = []
  for a, b in zip(vinvl_entry['box_features'], vinvl_entry['spatial_features']):
    object_feature = torch.cat((a, b))
    feature_list.append(object_feature)
  return torch.stack(feature_list)


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
  """Truncates a sequence pair in place to the maximum length."""

  # This is a simple heuristic which will always truncate the longer sequence
  # one token at a time. This makes more sense than truncating an equal percent
  # of tokens from each, since if one sequence is very short then each token
  # that's truncated likely contains more information than a longer sequence.
  while True:
      total_length = len(tokens_a) + len(tokens_b)
      if total_length <= max_length:
          break
      if len(tokens_a) > len(tokens_b):
          tokens_a.pop()
      else:
          tokens_b.pop()

#args.max_seq_length = 128
def tensorize_example(tokenizer, text_a, text_b, img_feat,  cls_token_at_end=False, pad_on_left=False,
                cls_token='[CLS]', sep_token='[SEP]', pad_token=0,
                sequence_a_segment_id=0, sequence_b_segment_id=1,
                cls_token_segment_id=1, pad_token_segment_id=0,
                mask_padding_with_zero=True):
    max_seq_length = 128
    max_img_seq_length = 40
    tokens_a = tokenizer.tokenize(text_a)

    tokens_b = None
    if text_b:
        tokens_b = tokenizer.tokenize(text_b)
        # Modifies `tokens_a` and `tokens_b` in place so that the total length is less than the specified length.
        # Account for [CLS], [SEP], [SEP] with "- 3"
        _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
    else:
        # Account for [CLS] and     [SEP] with "- 2"
        if len(tokens_a) > max_seq_length - 2:
            tokens_a = tokens_a[:(max_seq_length - 2)]

    tokens = tokens_a + [sep_token]
    segment_ids = [sequence_a_segment_id] * len(tokens)

    if tokens_b:
        tokens += tokens_b + [sep_token]
        segment_ids += [sequence_b_segment_id] * (len(tokens_b) + 1)

    if cls_token_at_end:
        tokens = tokens + [cls_token]
        segment_ids = segment_ids + [cls_token_segment_id]
    else:
        tokens = [cls_token] + tokens
        segment_ids = [cls_token_segment_id] + segment_ids

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real tokens are attended to.
    input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

    # Zero-pad up to the sequence length.
    padding_length = max_seq_length - len(input_ids)
    if pad_on_left:
        input_ids = ([pad_token] * padding_length) + input_ids
        input_mask = ([0 if mask_padding_with_zero else 1] * padding_length) + input_mask
        segment_ids = ([pad_token_segment_id] * padding_length) + segment_ids
    else:
        input_ids = input_ids + ([pad_token] * padding_length)
        input_mask = input_mask + ([0 if mask_padding_with_zero else 1] * padding_length)
        segment_ids = segment_ids + ([pad_token_segment_id] * padding_length)


    # image features
    
    if img_feat:
      if img_feat.shape[0] > max_img_seq_length:
          img_feat = img_feat[0: max_img_seq_length, ]
          if max_img_seq_length > 0:
              input_mask = input_mask + [1 if mask_padding_with_zero else 0] * img_feat.shape[0]
              segment_ids += [sequence_b_segment_id] * img_feat.shape[0]
      else:
          if max_img_seq_length > 0:
              input_mask = input_mask + [1 if mask_padding_with_zero else 0] * img_feat.shape[0]
              segment_ids = segment_ids + [sequence_b_segment_id] * img_feat.shape[0]
          padding_matrix = torch.zeros((max_img_seq_length - img_feat.shape[0], img_feat.shape[1]))
          img_feat = torch.cat((img_feat, padding_matrix), 0)
          if max_img_seq_length > 0:
              input_mask = input_mask + ([0 if mask_padding_with_zero else 1] * padding_matrix.shape[0])
              segment_ids = segment_ids + [pad_token_segment_id] * padding_matrix.shape[0]

 

      return (torch.tensor(input_ids, dtype=torch.long).unsqueeze(0),
            torch.tensor(input_mask, dtype=torch.long).unsqueeze(0),
            torch.tensor(segment_ids, dtype=torch.long)[0:128].unsqueeze(0),
            img_feat.unsqueeze(0))
    else:
      return (torch.tensor(input_ids, dtype=torch.long).unsqueeze(0),
            torch.tensor(input_mask, dtype=torch.long).unsqueeze(0),
            torch.tensor(segment_ids, dtype=torch.long)[0:128].unsqueeze(0),
            )





def save_oscar_output(vinvl_output_file_path, greater_save_directory, dataset_labelmap, tokenizer, device):
  vinvl_outputs_dict = torch.load(vinvl_output_file_path)
  oscar_outputs_dict = {}
  n = 0
  for dict_id in list(vinvl_outputs_dict.keys()):
    if n%100 == 0:
      print(n)

    n+=1
    vinvl_entry = vinvl_outputs_dict[dict_id]
    text_a = vinvl_entry['caption']
    text_b = get_objects(vinvl_entry, dataset_labelmap)
    img_feats = get_img_features(vinvl_entry)
    input = tensorize_example(tokenizer, text_a, text_b, img_feats,
                             cls_token_at_end=False, pad_on_left=False,
                cls_token='[CLS]', sep_token='[SEP]', pad_token=0,
                sequence_a_segment_id=0, sequence_b_segment_id=1,
                cls_token_segment_id=1, pad_token_segment_id=0,
                mask_padding_with_zero=True)
    outputs = model(input_ids=input[0].to(device), attention_mask = input[1].to(device), token_type_ids=input[2].to(device), img_feats=input[3].to(device))
    output1 = outputs[0].to('cpu').squeeze(0)[0]
    output2 = outputs[1].to('cpu')
    outputs = {}
    outputs['hateful'] = vinvl_entry['hateful']
    outputs['vector1'] = output1
    outputs['vector2'] = output2
    oscar_outputs_dict[dict_id] = outputs
  num = [int(i) for i in list(vinvl_output_file_path) if i.isdigit()]
  num = ''.join([str(elem) for elem in num])
  output_path = os.path.join(greater_save_directory, 'oscar_tensors_'+num+'.pth')
  torch.save(oscar_outputs_dict, output_path)


In [ ]:
import os
import json

from transformers.pytorch_transformers import BertTokenizer

import torch_xla.core.xla_model as xm

device = xm.xla_device()

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

greater_save_directory = 'path_to_save_test_set_output_finetuned_oscar_tensor'
labelmap_file = 'path_to_vinvl_labelmap_json'
dataset_allmap = json.load(open(labelmap_file, 'r'))
dataset_labelmap = {int(val): key for key, val in dataset_allmap['label_to_idx'].items()}

greater_vinvl_path = 'path_to_test_vinvl_tensors'
for vinvl_names in os.listdir(greater_vinvl_path):
  vinvl_output_file_path = os.path.join(greater_vinvl_path, vinvl_names)
  print('***************************************')
  print(vinvl_output_file_path)
  save_oscar_output(vinvl_output_file_path, greater_save_directory, dataset_labelmap, tokenizer, device)
  print()
  print()